In [1]:
import pandas as pd
import math

### Jeu de boules

Dans un sac se trouvent des boules rouges, vertes et bleues. Jouons.

Le but ultime du jeu (pas le notre) est de déterminer la quantité de boules contenue dans le sac. Pour se faire, notre elfe maître du jeu va révéler plusieurs fois (nombre aléatoire) une certaine quantité (aléatoire) de boules du sac.

Les résultats des tirages sont référencés dans le fichier input.txt, nous souhaitons isoler les parties possibles sous la condition suivante : "Le sac contient 12 boules rouges, 13 boules vertes et 14 boules bleues" puis obtenir la somme des IDs de ces dernières.

Conditionnement de l'impossibilité d'une partie :

    - Une partie est impossible si l'elfe tire plus de 39 boules strictement (faible), utile pour une potentielle "optimisation"
    - Une partie est impossible si l'efle tire plus de 12 boules rouges strictement, ou plus de 13 boules vertes strictement, ou plus de 14 boules bleues strictement

#### Récupèrer l'information


In [2]:
# Lecture
with open('input.txt', 'r') as file:
    text = file.read()

#### Transformer l'information

Je sais pas pourquoi mais j'avais envie de mettre en forme des DataFrames pour ensuite faire des calculs dessus.

In [3]:
# Un string pour chaque game
rows = text.strip().split('\n')

In [4]:
rows[0]

'Game 1: 7 green, 14 red, 5 blue; 8 red, 4 green; 6 green, 18 red, 9 blue'

In [5]:
# Parsing d'une ligne pour récupèrer l'information sous une forme exploitable (tuple)

def parsing_line(line):
    
    parts = line.strip().split(': ')
    id_partie = parts[0]
    data_partie = parts[1].split('; ')
    
    return id_partie, data_partie


In [6]:
parsing_line(rows[0])

('Game 1',
 ['7 green, 14 red, 5 blue', '8 red, 4 green', '6 green, 18 red, 9 blue'])

In [7]:
liste_data = []
for row in rows :
    id_partie, data_partie = parsing_line(row)
    liste_data.append((id_partie, data_partie))

In [8]:
liste_data[51]

('Game 52',
 ['1 blue',
  '5 green, 9 red',
  '2 blue, 1 green, 11 red',
  '2 blue, 13 red, 5 green',
  '6 green, 1 blue, 9 red'])

In [9]:
# Création du DataFrame
df = pd.DataFrame(columns = ['ID'])

In [10]:
for id_partie, data_partie in liste_data:
    dict_partie = {}
    for i , tirage in enumerate(data_partie): # On récupère les informations relatives aux couleurs
        couleurs = tirage.split(', ')
        
        rouge, vert, bleu = 0, 0, 0
        for couleur in couleurs : # On va formater l'information en tuples (r,g,b)
            nb, nom_couleur = couleur.split()
            if nom_couleur == 'red':
                rouge = int(nb)
            elif nom_couleur == 'green':
                vert = int(nb)
            elif nom_couleur == 'blue':
                bleu = int(nb)
        
        dict_partie[f'Tirage {i+1}'] = (rouge, vert, bleu)
    
    df = df.append({'ID' : id_partie, **dict_partie}, ignore_index= True)
        

C:\Users\Alexandre\AppData\Local\Temp\ipykernel_12904\1405919454.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'ID' : id_partie, **dict_partie}, ignore_index= True)
C:\Users\Alexandre\AppData\Local\Temp\ipykernel_12904\1405919454.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'ID' : id_partie, **dict_partie}, ignore_index= True)
C:\Users\Alexandre\AppData\Local\Temp\ipykernel_12904\1405919454.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'ID' : id_partie, **dict_partie}, ignore_index= True)
C:\Users\Alexandre\AppData\Local\Temp\ipykernel_12904\1405919454.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future 

In [11]:
df.head()

,ID,Tirage 1,Tirage 2,Tirage 3,Tirage 4,Tirage 5,Tirage 6
0,Game 1,"(14, 7, 5)","(8, 4, 0)","(18, 6, 9)",NaN,NaN,NaN
1,Game 2,"(15, 5, 3)","(14, 5, 1)","(11, 0, 0)","(3, 4, 1)","(0, 4, 1)","(10, 1, 0)"
2,Game 3,"(3, 11, 0)","(0, 4, 15)","(2, 10, 14)","(1, 3, 10)",NaN,NaN
3,Game 4,"(6, 1, 11)","(12, 0, 3)","(14, 1, 8)","(7, 0, 3)","(5, 0, 8)","(7, 1, 0)"
4,Game 5,"(3, 14, 3)","(2, 1, 1)","(1, 8, 3)","(1, 15, 8)",NaN,NaN


Petites remarques, ce parsing est robuste aux permutations qu'il peut y avoir dans le recensement des résultats. En outre, sur nos 100 parties, le nombres maximal de révélations de l'elfe est 6.

La colonne ID est pas vraiment utile ici (nous avons l'index, auquel il faudra faire attention quand on opère sur le dataframe), on va la supprimer.

In [12]:
df= df.drop(['ID'], axis = 1)

In [13]:
df.head()

,Tirage 1,Tirage 2,Tirage 3,Tirage 4,Tirage 5,Tirage 6
0,"(14, 7, 5)","(8, 4, 0)","(18, 6, 9)",NaN,NaN,NaN
1,"(15, 5, 3)","(14, 5, 1)","(11, 0, 0)","(3, 4, 1)","(0, 4, 1)","(10, 1, 0)"
2,"(3, 11, 0)","(0, 4, 15)","(2, 10, 14)","(1, 3, 10)",NaN,NaN
3,"(6, 1, 11)","(12, 0, 3)","(14, 1, 8)","(7, 0, 3)","(5, 0, 8)","(7, 1, 0)"
4,"(3, 14, 3)","(2, 1, 1)","(1, 8, 3)","(1, 15, 8)",NaN,NaN


In [14]:
type(df.iloc[0][5])

float

#### Isolement des parties impossibles

In [15]:
liste_index_impossibles = []

for index, row in df.iterrows():
    
    for col in row.index:
        
        tirage = row[col]
        if type(tirage) == float:
            break
        else :
            if tirage[0] > 12 or tirage[1] > 13 or tirage[2] > 14:
                liste_index_impossibles.append(index)
                break
        
        
        
        
        
        

In [16]:
liste_index_impossibles


[0,
 1,
 2,
 3,
 4,
 7,
 9,
 12,
 13,
 14,
 15,
 16,
 17,
 23,
 24,
 25,
 27,
 30,
 33,
 35,
 37,
 40,
 44,
 45,
 47,
 48,
 49,
 51,
 52,
 53,
 55,
 58,
 59,
 60,
 61,
 64,
 67,
 70,
 71,
 72,
 73,
 75,
 77,
 79,
 81,
 82,
 83,
 85,
 88,
 90,
 92,
 93,
 94,
 95,
 96,
 97,
 99]

In [17]:
# Récupération des informations relatives aux parties possibles
df_possible = df.loc[~df.index.isin(liste_index_impossibles)]

In [18]:
df_possible.shape

(43, 6)

In [19]:
df_possible.head()

,Tirage 1,Tirage 2,Tirage 3,Tirage 4,Tirage 5,Tirage 6
5,"(5, 8, 4)","(7, 9, 10)","(10, 7, 11)","(8, 9, 6)",NaN,NaN
6,"(9, 5, 11)","(6, 2, 12)","(8, 3, 4)","(8, 7, 9)","(5, 8, 0)",NaN
8,"(6, 7, 8)","(3, 10, 2)","(6, 7, 10)","(11, 5, 7)","(10, 11, 0)",NaN
10,"(6, 11, 4)","(12, 5, 1)","(7, 0, 1)","(11, 2, 3)","(6, 7, 2)",NaN
11,"(8, 1, 3)","(2, 3, 0)","(5, 1, 2)",NaN,NaN,NaN


In [20]:
sum(df_possible.index)

2058

OK on avait fait une erreur ! C'est parce qu'on a oublié que l'indexation commence à 1 dans notre cas d'utilisation, et pas à 0. Il faut donc ajouter une unité à notre réponse précédente pour chaque partie valide.

In [21]:
sum(df_possible.index) + df_possible.shape[0]

2101